In [ ]:
!pip install transformers

In [27]:
import torch
import numpy as np
from transformers import AutoModelForSequenceClassification
from transformers import BertTokenizerFast

tokenizer = BertTokenizerFast.from_pretrained('blanchefort/rubert-base-cased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment', return_dict=True)


def predict(text):
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt')
    outputs = model(**inputs)
    predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
    predicted = torch.argmax(predicted, dim=1).numpy()
    return predicted

Downloading:   0%|          | 0.00/499 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.40M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/943 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/712M [00:00<?, ?B/s]

In [50]:
import pandas as pd
import random
device = torch.device("cuda")
model.cuda()
DF = pd.read_csv('/content/women_with_predict.csv',encoding='utf-8',lineterminator='\n')[['text','label','model_predict']]

In [ ]:
for i in DF['text']:
  encoded_input = tokenizer(i, return_tensors='pt',padding=True, truncation=True, max_length=70)
  encoded_input = encoded_input.to(device)
  output = model(**encoded_input)
  scores = output[0][0].detach().cpu().numpy()
  scores = softmax(scores)
  k = scores.argmax()
  DF['model_predict'].iloc[tmp] = k


In [52]:
from sklearn.metrics import classification_report
print(classification_report(DF['model_predict'] ,DF['label']))

              precision    recall  f1-score   support

           0       0.54      0.72      0.61     20007
           1       0.87      0.91      0.89     28350
           2       0.88      0.68      0.77     36399

    accuracy                           0.77     84756
   macro avg       0.76      0.77      0.76     84756
weighted avg       0.80      0.77      0.77     84756

